In [1]:
import pandas as pd
import numpy as np
import nltk
import string

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df = pd.read_csv("./data/FAQ.csv")
df.head(100)

,questions,answer
0,Apa itu STT Cipasung?,Sekolah Tinggi Teknologi Cipasung (STT Cipasun...
1,Apa kepanjangan dari STT Cipasung?,STT Cipasung adalah singkatan dari Sekolah Tin...
2,Kapan STT Cipasung didirikan secara resmi?,STT Cipasung didirikan pada tahun 1997.
3,Siapa yang memberikan bantuan dan arahan dalam...,Pendirian STT Cipasung dilakukan dengan bantua...
4,Apa tujuan utama pendirian STT Cipasung?,Tujuannya adalah untuk menjawab tantangan dan ...
...,...,...
90,Di mana mahasiswa dapat mengurus pendaftaran s...,"""Pengurusan pendaftaran skripsi dilakukan mel..."
91,Kapan jadwal pendaftaran skripsi ?,"""Jadwal pendaftaran skripsi mengikuti ketentu..."
92,Apakah ada ketentuan mengenai pendaftaran skri...,"""Mahasiswa harus memenuhi prasyarat akademik ..."
93,Saya ingin mengetahui informasi tentang pendaf...,"""Informasi pendaftaran skripsi tersedia pada ..."


In [3]:
nltk.download('punkt')
nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('indonesian'))

def preprocess(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = word_tokenize(text)
    tokens = [t for t in tokens if t not in stop_words]
    return " ".join(tokens)

df['clean_question'] = df['questions'].apply(preprocess)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\fahmianoor\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\fahmianoor\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df['clean_question'])

In [5]:
def chatbot_response(user_input):
    user_input = preprocess(user_input)
    user_vector = vectorizer.transform([user_input])

    similarity = cosine_similarity(user_vector, tfidf_matrix)
    idx = similarity.argmax()
    score = similarity[0][idx]

    if score < 0.2:
        return "Maaf, saya tidak memahami pertanyaan Anda."
    else:
        return df.iloc[idx]['answer']

In [ ]:
from ipywidgets import Text, Button, Output
from IPython.display import display

input_box = Text(
    description='Pertanyaan:',
    placeholder='Ketik pertanyaan di sini...',
    layout={'width': '600px'}
)

button = Button(description='Kirim')
output = Output()

def on_click(b):
    with output:
        output.clear_output()
        print("Chatbot:", chatbot_response(input_box.value))

button.on_click(on_click)

display(input_box, button, output)


Text(value='', description='Pertanyaan:', layout=Layout(width='600px'), placeholder='Ketik pertanyaan di sini.…

Button(description='Kirim', style=ButtonStyle())

Output()